In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import pandas_ta as ta
import statsmodels.api as sm
import yfinance as yf
import numpy as np

In [3]:
# Downloading data
SPX = yf.download('^GSPC', start = '2005-01-01', interval = '1d')
VIX = yf.download('^VIX', start = SPX.index[1], interval = '1d')

# Calculating log returns
SPX['log_returns'] = np.log(SPX['Adj Close']) - np.log(SPX['Adj Close'].shift(1))
SPX['RSI'] = ta.rsi(SPX['Adj Close'])
SPX = SPX.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [18]:
variables = pd.DataFrame()
variables['VIX t + 5'] = VIX['Close'].shift(-5)
variables['VIX'] = VIX['Close']
variables['RSI'] = SPX['RSI']
variables['Volume'] = SPX['Volume']
variables = variables.dropna()

In [19]:

X = variables.iloc[:, 1:]
y = variables['VIX t + 5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_train_ = sm.add_constant(X_train)
x_test_ = sm.add_constant(X_test)

lr_sm = sm.OLS(y_train, x_train_).fit()

y_pred_train = lr_sm.predict(x_train_)

y_pred_test = lr_sm.predict(x_test_)

# Mostra as estatísticas 
print(lr_sm.summary())

                            OLS Regression Results                            
Dep. Variable:              VIX t + 5   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.851
Method:                 Least Squares   F-statistic:                     7173.
Date:                Sat, 07 Oct 2023   Prob (F-statistic):               0.00
Time:                        17:49:55   Log-Likelihood:                -10027.
No. Observations:                3763   AIC:                         2.006e+04
Df Residuals:                    3759   BIC:                         2.009e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5035      0.410     -3.665      0.0

In [23]:
resultados = pd.DataFrame({'Real': y_test, 'Previsto': y_pred_test})
resultados['RSI'] = x_test_['RSI']
resultados

,Real,Previsto,RSI
Date,,,
2023-06-05,15.010000,15.532513,64.564205
2018-11-21,18.790001,19.868331,37.281005
2010-03-12,16.969999,18.650746,69.801356
2019-11-18,11.870000,13.733607,74.371745
2009-03-30,40.930000,43.785669,51.265240
...,...,...,...
2006-10-02,11.680000,13.002393,62.070325
2012-04-04,17.200001,16.731005,54.250292
2007-06-20,15.530000,14.744711,49.427281
